# Telefonbuch-Client

Willkommen beim Telefonbuch-Client. Dieses Notebook ermöglicht es Ihnen, mit dem Telefonbuch-Server zu interagieren.

**Wichtiger Hinweis:** Stellen Sie sicher, dass das `Server.ipynb`-Notebook gestartet wurde und dessen Code-Zelle **aktiv läuft**, bevor Sie die Funktionen hier verwenden.

**Anleitung:**
Führen Sie die untere Code-Zelle aus. Der Code ist standardmäßig ausgeblendet. Die komplette Benutzeroberfläche inklusive des Log-Fensters wird dann angezeigt.

In [ ]:
import clientserver
import ipywidgets as widgets
from IPython.display import display, clear_output
import logging

# --- 1. Setup für das Logging --- 

log_widget_client = widgets.Output(layout={'border': '1px solid black', 'height': '200px', 'overflow_y': 'scroll'})

class WidgetHandler(logging.Handler):
    def __init__(self, widget):
        super().__init__()
        self.widget = widget

    def emit(self, record):
        with self.widget:
            print(self.format(record))

logger = logging.getLogger("vs2lab")
logger.setLevel(logging.INFO)

logger.handlers.clear()
logging.getLogger().handlers.clear()

logger.propagate = False

# Den neuen, einzigen Widget-Handler hinzufügen
handler = WidgetHandler(log_widget_client)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# --- 2. Definition der UI-Elemente und Callbacks ---

# Widgets für die 'GET'-Anfrage
name_input = widgets.Text(description="Name:", placeholder="z.B. jack")
search_button = widgets.Button(description="Suchen", button_style='primary')
result_output_get = widgets.Output()

def on_search_button_clicked(b):
    result_output_get.clear_output()
    with result_output_get:
        name = name_input.value.strip()
        if not name:
            print("Fehler: Bitte geben Sie einen Namen ein.")
            return
        
        print(f"Suche nach '{name}'...")
        try:
            client = clientserver.Client()
            response = client.call(f'GET\n{name}')
            clear_output(wait=True)
            print(f"Ergebnis: {response}")
        except ConnectionRefusedError:
            clear_output(wait=True)
            print("Verbindung fehlgeschlagen. Läuft der Server?")
        except Exception as e:
            clear_output(wait=True)
            print(f"Ein unerwarteter Fehler ist aufgetreten: {e}")

search_button.on_click(on_search_button_clicked)
get_interface = widgets.VBox([
    widgets.HTML(value='<h2>1. Einzelnen Namen abfragen</h2><p>Geben Sie einen Namen in das Textfeld ein und klicken Sie auf \'Suchen\'.</p>'),
    name_input, 
    search_button, 
    result_output_get
])

# Widgets für die 'GETALL'-Anfrage
getall_button = widgets.Button(description="Alle Einträge anzeigen", button_style='success')
result_output_getall = widgets.Output(layout={'max_height': '300px', 'overflow_y': 'scroll', 'border': '1px solid lightgray', 'padding': '5px'})

def on_getall_button_clicked(b):
    result_output_getall.clear_output()
    with result_output_getall:
        print("Alle Einträge werden vom Server angefordert...")
        try:
            client = clientserver.Client()
            response = client.call('GETALL')
            clear_output(wait=True)
            print("Telefonbuch-Einträge:")
            print(response.replace(';', '\n'))
        except ConnectionRefusedError:
            clear_output(wait=True)
            print("Verbindung fehlgeschlagen. Läuft der Server?")
        except Exception as e:
            clear_output(wait=True)
            print(f"Ein unerwarteter Fehler ist aufgetreten: {e}")

getall_button.on_click(on_getall_button_clicked)
getall_interface = widgets.VBox([
    widgets.HTML(value='<hr><h2>2. Alle Einträge anzeigen</h2><p>Klicken Sie auf den Button, um alle Einträge aus dem Telefonbuch aufzulisten.</p>'),
    getall_button, 
    result_output_getall
])

# Log-Anzeige-Interface
log_interface = widgets.VBox([
    widgets.HTML(value='<hr><h2>Client-Log</h2><p>Hier werden die technischen Log-Nachrichten des Clients angezeigt.</p>'),
    log_widget_client
])

# --- 3. Zusammenbau und Anzeige der gesamten UI ---
app_layout = widgets.VBox([
    get_interface,
    getall_interface,
    log_interface
])

display(app_layout)